<a href="https://colab.research.google.com/github/Bebhawna/Handwritten-Character-Recognition/blob/main/DDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import os
import cv2
import math

In [ ]:
base='/content/drive/MyDrive/Deepfake Detection/train_sample_videos'

In [ ]:
# # For better integrity and scalability and to ensure clealiness in code
# def get_filename_only(file):
#   file_basename=os.path.basename(file)
#   filename_only=file_basename.split('.')[0]
#   return filename_only
# with open(os.path.join(base,'metadata.json')) as metadata_json:
#   metadata=json.load(metadata_json)
#   print(len(metadata))

5


In [ ]:
# Creation of directory having mp4 extension video to process and capture the frame
for filename in metadata.keys():
    print(filename)
    if (filename.endswith(".mp4")):
        temporary_path = os.path.join(base, get_filename_only(filename))   #Attaching the file with the video and then creatign directory
        print('Creating Directory: ' + temporary_path)
        os.makedirs(temporary_path, exist_ok=True)
        print('Converting Video to Images...')
        count = 0
        video_file = os.path.join(base, filename)
        cap = cv2.VideoCapture(video_file)
        frame_rate = cap.get(5)
        #Initialising the loop to extract the frames from the video for better training
        while(cap.isOpened()):
            frame_id = cap.get(1) #current frame number
            ret, frame = cap.read()
            if (ret != True):
                break
                #Setting a particular limit in which the frame would be captured from the video
            if (frame_id % math.floor(frame_rate) == 0):
                print('Original Dimensions: ', frame.shape)
                # Rescaling the frames for better resolution
                if frame.shape[1] < 300:
                    scale_ratio = 2
                elif frame.shape[1] > 1900:
                    scale_ratio = 0.33
                elif frame.shape[1] > 1000 and frame.shape[1] <= 1900 :
                    scale_ratio = 0.5
                else:
                    scale_ratio = 1
                print('Scale Ratio: ', scale_ratio)
                # Redefining the dimensions of the image
                width = int(frame.shape[1] * scale_ratio)
                height = int(frame.shape[0] * scale_ratio)
                dim = (width, height)
                new_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
                print('Resized Dimensions: ', new_frame.shape)

                new_filename = '{}-{:03d}.png'.format(os.path.join(temporary_path, get_filename_only(filename)), count)  #Storing or overlapping the new images on old path
                count = count + 1
                cv2.imwrite(new_filename, new_frame)
        cap.release()  # Releasing the video when done
        print("Done!")
    else:
        continue

aagfhgtpmv.mp4
Creating Directory: /content/drive/MyDrive/Deepfake Detection/train_sample_videos/aagfhgtpmv
Converting Video to Images...
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920, 3)
Scale Ratio:  0.33
Resized Dimensions:  (356, 633, 3)
Original Dimensions:  (1080, 1920,

In [ ]:
!pip install mtcnn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.6 MB/s eta 0:00:00


In [ ]:
import cv2
from mtcnn import MTCNN
import sys, os.path
import json
from keras import backend as K
import tensorflow as tf

In [ ]:
base='/content/drive/MyDrive/Deepfake Detection/train_sample_videos'

In [ ]:
#To get the name of the file instead of complete path
def get_filename_only(file_path):
    file_basename = os.path.basename(file_path)
    filename_only = file_basename.split('.')[0]
    return filename_only

with open(os.path.join(base, 'metadata.json')) as metadata_json:
    metadata = json.load(metadata_json)
    print(len(metadata))

for filename in metadata.keys():
    temp_path = os.path.join(base, get_filename_only(filename))
    print('Processing Directory: ' + temp_path)
    if not os.path.exists(temp_path):
           print(f"Directory does not exist: {temp_path}. Extracting frames from video...")
           continue
    frame_images = [x for x in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, x))]
    faces_path = os.path.join(temp_path, 'faces')
    print('Creating Directory: ' + faces_path)
    os.makedirs(faces_path, exist_ok=True)
    print('Cropping Faces from Images...')

    for frame in frame_images:
        print('Processing ', frame)
        detector = MTCNN()
        image = cv2.cvtColor(cv2.imread(os.path.join(temp_path, frame)), cv2.COLOR_BGR2RGB)
        results = detector.detect_faces(image)
        print('Face Detected: ', len(results))
        count = 0

        for result in results:
            bounding_box = result['box']
            print(bounding_box)
            confidence = result['confidence']
            print(confidence)
            if len(results) < 2 or confidence > 0.95:
                margin_x = bounding_box[2] * 0.3  # 30% as the margin
                margin_y = bounding_box[3] * 0.3  # 30% as the margin
                x1 = int(bounding_box[0] - margin_x)
                if x1 < 0:
                    x1 = 0
                x2 = int(bounding_box[0] + bounding_box[2] + margin_x)
                if x2 > image.shape[1]:
                    x2 = image.shape[1]
                y1 = int(bounding_box[1] - margin_y)
                if y1 < 0:
                    y1 = 0
                y2 = int(bounding_box[1] + bounding_box[3] + margin_y)
                if y2 > image.shape[0]:
                    y2 = image.shape[0]
                print(x1, y1, x2, y2)
                crop_image = image[y1:y2, x1:x2]
                new_filename = '{}-{:02d}.png'.format(os.path.join(faces_path, get_filename_only(frame)), count)
                count = count + 1
                cv2.imwrite(new_filename, cv2.cvtColor(crop_image, cv2.COLOR_RGB2BGR))
            else:
                print('Skipped a face..')


5
Processing Directory: /content/drive/MyDrive/Deepfake Detection/train_sample_videos/aagfhgtpmv
Creating Directory: /content/drive/MyDrive/Deepfake Detection/train_sample_videos/aagfhgtpmv/faces
Cropping Faces from Images...
Processing  aagfhgtpmv-000.png
Face Detected:  1
[110, 58, 71, 92]
0.9999967813491821
88 30 202 177
Processing  aagfhgtpmv-001.png
Face Detected:  1
[167, 58, 72, 95]
0.9999997615814209
145 29 260 181
Processing  aagfhgtpmv-002.png
Face Detected:  1
[202, 60, 66, 94]
0.9998657703399658
182 31 287 182
Processing  aagfhgtpmv-003.png
Face Detected:  1
[319, 71, 65, 91]
0.9997005462646484
299 43 403 189
Processing  aagfhgtpmv-004.png
Face Detected:  1
[368, 52, 71, 98]
0.9994237422943115
346 22 460 179
Processing  aagfhgtpmv-005.png
Face Detected:  1
[340, 62, 61, 86]
0.9994107484817505
321 36 419 173
Processing  aagfhgtpmv-006.png
Face Detected:  1
[350, 43, 63, 86]
0.9989352822303772
331 17 431 154
Processing  aagfhgtpmv-007.png
Face Detected:  1
[302, 36, 67, 87]
0

In [ ]:
!pip install split-folders


In [ ]:
import json
import os
from distutils.dir_util import copy_tree
import shutil
import numpy as np
import splitfolders

In [ ]:

base_path = '/content/drive/MyDrive/Deepfake Detection/train_sample_videos'
dataset_path = '/content/drive/MyDrive/Deepfake Detection/prepared_dataset'
print('Creating Directory: ' + dataset_path)
os.makedirs(dataset_path, exist_ok=True)

Creating Directory: /content/drive/MyDrive/Deepfake Detection/prepared_dataset


In [ ]:
tmp_fake_path = '/content/drive/MyDrive/Deepfake Detection/tmp_fake_faces'
print('Creating Directory: ' + tmp_fake_path)
os.makedirs(tmp_fake_path, exist_ok=True)

def get_filename_only(file_path):
    file_basename = os.path.basename(file_path)
    filename_only = file_basename.split('.')[0]
    return filename_only

with open(os.path.join(base_path, 'metadata.json')) as metadata_json:
    metadata = json.load(metadata_json)
    print(len(metadata))

real_path = os.path.join(dataset_path, 'real')
print('Creating Directory: ' + real_path)
os.makedirs(real_path, exist_ok=True)

fake_path = os.path.join(dataset_path, 'fake')
print('Creating Directory: ' + fake_path)
os.makedirs(fake_path, exist_ok=True)

for filename in metadata.keys():
    print(filename)
    print(metadata[filename]['label'])
    tmp_path = os.path.join(os.path.join(base_path, get_filename_only(filename)), 'faces')
    print(tmp_path)
    if os.path.exists(tmp_path):
        if metadata[filename]['label'] == 'REAL':
            print('Copying to :' + real_path)
            copy_tree(tmp_path, real_path)
        elif metadata[filename]['label'] == 'FAKE':
            print('Copying to :' + tmp_fake_path)
            copy_tree(tmp_path, tmp_fake_path)
        else:
            print('Ignored..')

all_real_faces = [f for f in os.listdir(real_path) if os.path.isfile(os.path.join(real_path, f))]
print('Total Number of Real faces: ', len(all_real_faces))

all_fake_faces = [f for f in os.listdir(tmp_fake_path) if os.path.isfile(os.path.join(tmp_fake_path, f))]
print('Total Number of Fake faces: ', len(all_fake_faces))

random_faces = np.random.choice(all_fake_faces, len(all_real_faces), replace=False)
for fname in random_faces:
    src = os.path.join(tmp_fake_path, fname)
    dst = os.path.join(fake_path, fname)
    shutil.copyfile(src, dst)

print('Down-sampling Done!')

# Split into Train/ Val/ Test folders
splitfolders.ratio(dataset_path, output='split_dataset', seed=1377, ratio=(.8, .1, .1)) # default values
print('Train/ Val/ Test Split Done!')

Creating Directory: /content/drive/MyDrive/Deepfake Detection/tmp_fake_faces
5
Creating Directory: /content/drive/MyDrive/Deepfake Detection/prepared_dataset/real
Creating Directory: /content/drive/MyDrive/Deepfake Detection/prepared_dataset/fake
aagfhgtpmv.mp4
FAKE
/content/drive/MyDrive/Deepfake Detection/train_sample_videos/aagfhgtpmv/faces
Copying to :/content/drive/MyDrive/Deepfake Detection/tmp_fake_faces
aapnvogymq.mp4
FAKE
/content/drive/MyDrive/Deepfake Detection/train_sample_videos/aapnvogymq/faces
Copying to :/content/drive/MyDrive/Deepfake Detection/tmp_fake_faces
abarnvbtwb.mp4
REAL
/content/drive/MyDrive/Deepfake Detection/train_sample_videos/abarnvbtwb/faces
Copying to :/content/drive/MyDrive/Deepfake Detection/prepared_dataset/real
abofeumbvv.mp4
FAKE
/content/drive/MyDrive/Deepfake Detection/train_sample_videos/abofeumbvv/faces
Copying to :/content/drive/MyDrive/Deepfake Detection/tmp_fake_faces
abqwwspghj.mp4
FAKE
/content/drive/MyDrive/Deepfake Detection/train_sample

Copying files: 35 files [00:00, 240.43 files/s]

Train/ Val/ Test Split Done!


In [ ]:

from distutils.dir_util import copy_tree
import shutil
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
print('TensorFlow version: ', tf.__version__)

TensorFlow version:  2.17.1


In [ ]:
dataset_path = '/content/drive/MyDrive/Deepfake Detection/split_dataset'

In [ ]:
tmp_debug_path = '/content/drive/MyDrive/Deepfake Detection/tmp_debug'
print('Creating Directory: ' + tmp_debug_path)
os.makedirs(tmp_debug_path, exist_ok=True)


Creating Directory: /content/drive/MyDrive/Deepfake Detection/tmp_debug


In [ ]:
def get_filename_only(file_path):
    file_basename = os.path.basename(file_path)
    filename_only = file_basename.split('.')[0]
    return filename_only

In [ ]:
!pip install efficientnet

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
input_size=128
batch_size_num=32
train_path=os.path.join(dataset_path,'train')
val_path=os.path.join(dataset_path,'val')
test_path=os.path.join(dataset_path,'test')

In [ ]:
train_datagen=ImageDataGenerator(
    rescale= 1/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
)
train_generator= train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(input_size,input_size),
    color_mode='rgb',
    class_mode="binary",
    batch_size=batch_size_num,
    shuffle=True
)
val_datagen=ImageDataGenerator(
    rescale=1/255
)
val_generator=val_datagen.flow_from_directory(
    directory=val_path,
    target_size=(input_size,input_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size=batch_size_num,
    shuffle=True,
)
test_datagen=ImageDataGenerator(
    rescale=1/255
)
test_generator=test_datagen.flow_from_directory(
    directory=test_path,
    classes=['real','fake'],
    target_size=(input_size, input_size),
    color_mode="rgb",
    class_mode=None,
    batch_size=1,
    shuffle=False
)

Found 16 images belonging to 2 classes.
Found 2 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [ ]:
efficient_net = EfficientNetB0(
    weights = 'imagenet',
    input_shape = (input_size, input_size, 3),
    include_top = False,
    pooling = 'max'
)

model = Sequential()
model.add(efficient_net)
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.summary()
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnet-b0 (Functional)         │ (None, 1280)                │       4,049,564 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,771,229 (18.20 MB)

 Trainable params: 4,729,213 (18.04 MB)

 Non-trainable params: 42,016 (164.12 KB)

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_filepath = '/content/drive/MyDrive/Deepfake Detection/tmp_checkpoint'
print('Creating Directory: ' + checkpoint_filepath)
os.makedirs(checkpoint_filepath, exist_ok=True)

custom_callbacks = [
    EarlyStopping(
        monitor = 'val_loss',
        mode = 'min',
        patience = 5,
        verbose = 1
    ),
    ModelCheckpoint(
        filepath = os.path.join(checkpoint_filepath, 'best_model.keras'),
        monitor = 'val_loss',
        mode = 'min',
        verbose = 1,
        save_best_only = True
    )
]

Creating Directory: /content/drive/MyDrive/Deepfake Detection/tmp_checkpoint


In [ ]:
num_epochs = 30
history = model.fit(
    train_generator,
    epochs = num_epochs,
    steps_per_epoch = len(train_generator),
    validation_data = val_generator,
    validation_steps = len(val_generator),
    callbacks = custom_callbacks
)
print(history.history)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40s/step - accuracy: 1.0000 - loss: 0.0013
Epoch 1: val_loss improved from 0.01107 to 0.00971, saving model to /content/drive/MyDrive/Deepfake Detection/tmp_checkpoint/best_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 44s 44s/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 0.0097
Epoch 2/30


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/30


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 1.0000 - loss: 0.0015
Epoch 3: val_loss improved from 0.00971 to 0.00849, saving model to /content/drive/MyDrive/Deepfake Detection/tmp_checkpoint/best_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0085
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 1.0000 - loss: 8.3825e-04
Epoch 5: val_loss improved from 0.00849 to 0.00735, saving model to /content/drive/MyDrive/Deepfake Detection/tmp_checkpoint/best_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 1.0000 - loss: 8.3825e-04 - val_accuracy: 1.0000 - val_loss: 0.0074
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 1.0000 - loss: 3.8852e-04
Epoch 7: val_loss improved from 0.00735 to 0.00651, saving mode

In [ ]:
best_model = load_model(os.path.join(checkpoint_filepath, 'best_model.keras'))

In [ ]:
import matplotlib.pyplot as plt


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

train_epochs=range(len(acc))
val_epochs=range(len(val_acc))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# True labels and predicted labels
for images,labels in validation_ds:
  predicted_labels_batches=model.predict(images)
  predicted_labels_batches=tf.argmax(predicted_labels_batches,axis=1)
  test_labels.append(labels.numpy())
  predicted_labels.append(predicted_labels_batches.numpy())
test_labels=np.concatenate(test_labels)
predicted_labels=np.concatenate(predicted_labels)

if len(test_labels.shape)>1:
  test_labels=np.argmax(test_labels,axis=1)

# Accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Precision
precision = precision_score(true_labels, predicted_labels, pos_label='fake')

# Recall
recall = recall_score(true_labels, predicted_labels, pos_label='fake')

# F1 Score
f1 = f1_score(true_labels, predicted_labels, pos_label='fake')

# Confusion Matrix (optional, for verification)
cm = confusion_matrix(true_labels, predicted_labels, labels=['real', 'fake'])

print("Confusion Matrix:\n", cm)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")


Confusion Matrix:
 [[1 1]
 [0 4]]
Accuracy: 83.33%
Precision: 80.00%
Recall: 100.00%
F1 Score: 88.89%


In [ ]:
test_generator.reset()

In [ ]:
preds=best_model.predict(
    test_generator,
    verbose=1
)
test_result=pd.DataFrame({
    "Filename": test_generator.filenames,
    "Prediction": preds.flatten()
})
print(test_result)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
                     Filename  Prediction
0  real/abarnvbtwb-000-00.png    0.660621
1  real/abarnvbtwb-002-00.png    0.999703
2  fake/aagfhgtpmv-001-00.png    0.018093
3  fake/aagfhgtpmv-009-00.png    0.152356


In [ ]:
print(f"Filename: {filename}, Prediction: {pred[0]:.6f}, Classified as: {label}")

Filename: real/abarnvbtwb-000-00.png, Prediction: 0.660621, Classified as: real


In [ ]:
from google.colab import files
import cv2
import numpy as np
import os
import shutil
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Upload a file
uploaded = files.upload()

# Load the best model for prediction
checkpoint_filepath = '/content/drive/MyDrive/Deepfake Detection/tmp_checkpoint'
best_model_path = os.path.join(checkpoint_filepath, 'best_model.keras')
model = load_model(best_model_path)

# Function to preprocess and predict a single image
def preprocess_and_predict(image_path):
    image = load_img(image_path, target_size=(128, 128))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize the image

    prediction = model.predict(image)
    if prediction[0] < 0.5:
        return 'Fake', prediction[0]
    else:
        return 'Real', prediction[0]

# Function to extract frames from a video
def extract_frames_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = f"/tmp/frame_{frame_count:04d}.jpg"
        cv2.imwrite(frame_filename, frame)
        frames.append(frame_filename)
        frame_count += 1
    cap.release()
    return frames

# Handle uploaded files (Image or Video)
uploaded_file_path = list(uploaded.keys())[0]
file_extension = os.path.splitext(uploaded_file_path)[1].lower()

if file_extension in ['.jpg', '.jpeg', '.png']:
    print("File is an image. Processing image...")
    label, confidence = preprocess_and_predict(uploaded_file_path)
    print(f"Prediction: {label} with confidence {confidence[0]:.2f}") # Access the element using [0]
elif file_extension in ['.mp4', '.avi', '.mov']:
    print("File is a video. Extracting frames...")
    frames = extract_frames_from_video(uploaded_file_path)
    # Predict on the first frame
    label, confidence = preprocess_and_predict(frames[0])
    print(f"Prediction for the first frame: {label} with confidence {confidence[0]:.2f}") # Access the element using [0]
else:
    print("Unsupported file format. Please upload an image or video.")

Saving test1.png to test1 (6).png
File is an image. Processing image...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Fake with confidence 0.08
